In [1]:
#some basic imports and setups
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datagenerator import ImageDataGenerator
from datetime import datetime

#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

%matplotlib inline

In [5]:
from alexnet import AlexNet
from caffe_classes import class_names

tf.reset_default_graph()

# Network params
dropout_rate = 0.5
num_classes = 1000
train_layers = ['fc8', 'fc7']
batch_size = 128

#placeholder for input and dropout rate
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
model = AlexNet(x, keep_prob, num_classes, train_layers)

# Link variable to model output
score = model.fc8

In [10]:
# Test validation accuracy of fine-tuned AlexNet

# Create Validation data generator
val_file = './val.txt'
val_generator = ImageDataGenerator(val_file, shuffle = False)

# Params
learning_rate = 0.01
num_epochs = 20
batch_size = 128
checkpoint_path = "./tmp/finetune_alexnet/"

# Get the number of validation steps per epoch
val_batches_per_epoch = np.floor(val_generator.data_size / batch_size).astype(np.int32)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
  correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Saver to load checkpoint
saver = tf.train.Saver()

with tf.Session() as sess:
#     # Load the pretrained weights into the model [from BVLC]
#     model.load_initial_weights(sess)
    
    # Load model from checkpoint [fine-tuned AlexNet starting from BVLC]    
    saver.restore(sess, tf.train.latest_checkpoint(checkpoint_path))

    print("{} Start validation".format(datetime.now()))
    test_acc = 0.
    test_count = 0
    for _ in range(val_batches_per_epoch):
        
        batch_tx, batch_ty = val_generator.next_batch(batch_size)
        acc = sess.run(accuracy, feed_dict={x: batch_tx, 
                                            y: batch_ty, 
                                            keep_prob: 1.})
        test_acc += acc
        test_count += 1
        if test_count % 30 == 0:
            print(str(i) + ' out of ' + str(val_batches_per_epoch) + ' batches done')
            print("Accuracy: " + str(test_acc / test_count))
    test_acc /= test_count
    print("{} Validation Accuracy = {:.4f}".format(datetime.now(), test_acc))

INFO:tensorflow:Restoring parameters from ./tmp/finetune_alexnet/model_epoch2.ckpt
2017-07-13 21:01:05.129881 Start validation
6 out of 390 batches done
Accuracy: 0.470052083333
6 out of 390 batches done
Accuracy: 0.458854166667
6 out of 390 batches done
Accuracy: 0.461458333333
6 out of 390 batches done
Accuracy: 0.456380208333
6 out of 390 batches done
Accuracy: 0.45609375
6 out of 390 batches done
Accuracy: 0.456553819444
6 out of 390 batches done
Accuracy: 0.456770833333
6 out of 390 batches done
Accuracy: 0.460546875
6 out of 390 batches done
Accuracy: 0.456394675926
6 out of 390 batches done
Accuracy: 0.457057291667
6 out of 390 batches done
Accuracy: 0.459209280303
6 out of 390 batches done
Accuracy: 0.462934027778
6 out of 390 batches done
Accuracy: 0.460596955128
2017-07-13 21:14:37.359516 Validation Accuracy = 0.4606
